
# 01 – Vue d'ensemble du projet et données

Ce notebook présente la **structure générale** du projet, charge les données de base
et donne quelques statistiques descriptives.  
Il sert de point d'entrée pour comprendre rapidement ce que contient le dépôt.

**Objectifs :**
- comprendre l'architecture (`src/models`, `src/pricing`, `src/calibration`, `src/risk`, `notebooks`, etc.) ;
- explorer les données disponibles dans `data/` ;
- visualiser quelques séries temporelles et distributions de rendements.


In [2]:

import os
import pathlib
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# On suppose que ce notebook se trouve dans le dossier "notebooks"
NOTEBOOK_DIR = pathlib.Path().resolve()
PROJECT_ROOT = NOTEBOOK_DIR.parent

print("Notebook dir :", NOTEBOOK_DIR)
print("Project root :", PROJECT_ROOT)

# Pour pouvoir faire `import src.xxx`
if str(PROJECT_ROOT / "src") not in sys.path:
    sys.path.append(str(PROJECT_ROOT / "src"))


Notebook dir : /home/hugo/Bureau/Github/quant-simulations/notebooks
Project root : /home/hugo/Bureau/Github/quant-simulations



## 1. Architecture du projet

On liste récursivement les principaux répertoires pour avoir une vue d'ensemble.


In [3]:

for root, dirs, files in os.walk(PROJECT_ROOT):
    # On masque certains dossiers techniques
    dirs[:] = [d for d in dirs if d not in {'.git', '.pytest_cache', '__pycache__', '.idea'}]
    rel_root = pathlib.Path(root).relative_to(PROJECT_ROOT)
    indent = "  " * (len(rel_root.parts))
    print(f"{indent}{rel_root}/")
    for f in files:
        print(f"{indent}  - {f}")


./
  - requirements.txt
  - setup.cfg
  - plotting.py
  - .gitignore
  - README.md
  notebooks/
    - 01_overview.ipynb
    - 03_pricing_comparison.ipynb
    - 01_overview_and_data.ipynb
    - 03_euler_vs_exact.ipynb
    - 04_calibration_and_risk.ipynb
    - 02_validation.ipynb
    - 02_models_simulation.ipynb
  src/
    - simulation.py
    - __init__.py
    - stats.py
    - plotting.py
    - utils.py
    src/models/
      - __init__.py
      - heston.py
      - base_model.py
      - variance_gamma.py
      - gbm.py
    src/pricing/
      - monte_carlo.py
      - black_scholes.py
      - greeks.py
      - pde_solver.py
    src/calibration/
      - likelihood.py
      - optimization.py
      - moments.py
    src/risk/
      - var.py
  tests/
    - test_stats.py
  data/



## 2. Chargement des données

On parcourt le dossier `data/` et on charge quelques fichiers (CSV, Parquet, etc.)
pour inspecter rapidement les séries temporelles disponibles.


In [4]:

data_dir = PROJECT_ROOT / "data"
print("Fichiers dans data/:")
if data_dir.exists():
    for p in sorted(data_dir.glob("**/*")):
        if p.is_file():
            print(" -", p.relative_to(PROJECT_ROOT))
else:
    print("Aucun dossier data/ trouvé.")


Fichiers dans data/:


In [5]:

# Exemple : chargement du premier CSV trouvé (adapter si besoin)
sample_csv = None
for p in sorted(data_dir.glob("**/*.csv")):
    sample_csv = p
    break

if sample_csv is not None:
    print("Chargement de :", sample_csv)
    df = pd.read_csv(sample_csv)
    display(df.head())
    print(df.describe(include='all'))
else:
    print("Aucun CSV trouvé dans data/. Adapter ce bloc selon vos données.")


Aucun CSV trouvé dans data/. Adapter ce bloc selon vos données.



## 3. Rendements et statistiques de base

À partir d'une série de prix $P_t$, on calcule les **rendements logarithmiques**
$$ R_t = \log P_t - \log P_{t-1}. $$


In [6]:

if 'df' in globals():
    # On suppose qu'il y a une colonne 'Close' ou 'Price'
    price_col_candidates = [c for c in df.columns if c.lower() in ('close', 'price', 'adj_close')]
    if price_col_candidates:
        price_col = price_col_candidates[0]
        prices = df[price_col].astype(float).values
        log_returns = np.diff(np.log(prices))
        print("Nombre de rendements :", len(log_returns))
        print("Moyenne :", np.mean(log_returns))
        print("Volatilité (std) :", np.std(log_returns))
    else:
        print("Impossible de trouver une colonne de prix évidente dans df.")


In [7]:

if 'log_returns' in globals():
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))

    axes[0].plot(prices)
    axes[0].set_title("Série de prix")
    axes[0].set_xlabel("Temps")
    axes[0].set_ylabel("Prix")

    axes[1].hist(log_returns, bins=50, density=True)
    axes[1].set_title("Histogramme des rendements log")
    axes[1].set_xlabel("Rendement")
    axes[1].set_ylabel("Densité")

    plt.tight_layout()
    plt.show()



## 4. Points d'analyse possibles

- Comparer les distributions de rendements sur différentes fenêtres temporelles.
- Étudier la présence d'asymétrie / leptokurtose (moments d'ordre supérieur).
- Mettre en regard ces statistiques empiriques avec celles prédites par les modèles (GBM, Heston, Variance Gamma) dans les autres notebooks.
